In [1]:
from __future__ import print_function
from pyspark import SparkContext
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vectors
import logging, gensim, bz2
from django.utils.encoding import smart_str, smart_unicode

In [2]:
def removeUnderscores(text):
    return text.replace ("_", " ")    

In [3]:
import csv
import  urllib2
from SPARQLWrapper import SPARQLWrapper, JSON
docs =[]
with open("/Users/sumedhakhatter/Downloads/trainingData.txt") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        #  print (row['DBpediaURL'], row['categoriesList'])
        #read data from URL        
        p = row['DBpediaURL']
        #print(p)
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
        sparql.setQuery("""
        PREFIX dbp: <http://dbpedia.org/ontology/>
 
        SELECT ?abstract
        WHERE {
        %s dbp:abstract ?abstract.
         FILTER langMatches(lang(?abstract),'en')
         }
        """%p)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
       
        
        temp =""
        for result in results["results"]["bindings"]:
            temp = temp +result["abstract"]["value"]
            
       
        
        sparql.setQuery("""
        PREFIX dc: <http://purl.org/dc/terms/>
 
        SELECT ?subject
        WHERE {
        %s dc:subject ?subject.
         FILTER langMatches(lang(?subject),'en')
         }
        """%p)
        
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()

        for result in results["results"]["bindings"]:
            temp = temp + result["subject"]["value"]
        
       
        categoriesText = removeUnderscores(row['categoriesList'].decode('utf8'))
        docs.append(temp+ categoriesText) 
        #print(categoriesText)
        # print(docs)

In [4]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
import re

tokenizer = RegexpTokenizer(r'\w+')
en_stop = get_stop_words('en')

p_stemmer = PorterStemmer()


texts = []
        
for i in docs:
    raw = i.lower()
    tokens=tokenizer.tokenize(raw)
    stopped_tokens =[i for i in tokens if not i in en_stop]
   
    new_stopped_tokens = []
    #for token in stopped_tokens:
     #   new_stopped_tokens.append(re.sub(r'[^\w]','',token))
    #new_stopped_tokens = filter(None, new_stopped_tokens)    
    
    #print(new_stopped_tokens)
    print(stopped_tokens)
    for token in stopped_tokens:
        new_stopped_tokens.append(token.encode('utf8'))
        
        
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    texts.append(stemmed_tokens)
    
dictionary = corpora.Dictionary(texts)

corpus = [dictionary.doc2bow(text) for text in texts]

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word=dictionary, passes = 500)

[u'american', u'football', u'conference', u'afc', u'one', u'two', u'conferences', u'national', u'football', u'league', u'nfl', u'conference', u'counterpart', u'national', u'football', u'conference', u'nfc', u'currently', u'contain', u'16', u'teams', u'making', u'32', u'teams', u'nfl', u'current', u'afc', u'title', u'holder', u'new', u'england', u'patriots', u'national', u'football', u'league', u'american', u'football', u'league', u'organizations', u'established', u'1970']
[u'arizona', u'cardinals', u'professional', u'american', u'football', u'team', u'based', u'phoenix', u'arizona', u'metropolitan', u'area', u'cardinals', u'currently', u'members', u'west', u'division', u'national', u'football', u'conference', u'nfc', u'national', u'football', u'league', u'nfl', u'cardinals', u'founded', u'1898', u'oldest', u'continuously', u'run', u'professional', u'football', u'team', u'united', u'states', u'team', u'established', u'chicago', u'1898', u'charter', u'member', u'nfl', u'1920', u'along', 

In [5]:
ldamodel.print_topics()

[(0,
  u'0.015*bird + 0.012*leagu + 0.012*nfl + 0.011*team + 0.010*footbal + 0.010*s + 0.008*state + 0.008*american + 0.007*game + 0.006*first'),
 (1,
  u'0.010*aspirin + 0.010*bird + 0.008*isra + 0.008*sharon + 0.007*minist + 0.006*war + 0.006*s + 0.006*israel + 0.005*bankruptci + 0.005*australia'),
 (2,
  u'0.022*american + 0.012*peopl + 0.011*state + 0.008*lincoln + 0.007*unit + 0.007*s + 0.007*gate + 0.006*war + 0.006*russian + 0.006*polit'),
 (3,
  u'0.015*american + 0.012*citi + 0.010*london + 0.009*s + 0.006*film + 0.006*nova + 0.006*scotia + 0.006*tenni + 0.006*austrian + 0.005*unit'),
 (4,
  u'0.020*club + 0.020*footbal + 0.010*state + 0.009*cup + 0.007*leagu + 0.007*one + 0.006*australian + 0.006*beryllium + 0.005*establish + 0.005*sport')]

In [6]:
test = "Football is a family of team sports that involve, to varying degrees, kicking a ball with the foot to score a goal. Unqualified, the word football is understood to refer to whichever form of football is the most popular in the regional context in which the word appears. Sports commonly called 'football' in certain places include: association football (known as soccer in some countries); gridiron football (specifically American football or Canadian football); Australian rules football; rugby football (either rugby league or rugby union); and Gaelic football.[1][2] These different variations of football are known as football codes."
doc = test.split()
vec = dictionary.doc2bow(doc)
print(ldamodel[vec])

[(0, 0.12990502071899848), (1, 0.016944570225481851), (2, 0.0168432431507663), (3, 0.38465986813995495), (4, 0.45164729776479845)]
